In [6]:
import os
import csv
import statistics

# Directory containing experiment folders
experiments_dir = "experiments"

# Collect rows for each table
parameters_rows = []
summary_rows = []

# Track all parameter keys and AS@k columns
all_param_keys = set()
as_columns = set()

# Mapping from full CSV metric names to column labels
metrics_column_map = {
    "Average Best Loss": "Loss",
    "Std Best Loss": "Loss",
    "Average Gradient Time": "Grad (s)",
    "Std Gradient Time": "Grad (s)",
    "Average Sampling Time": "Sampling (s)",
    "Std Sampling Time": "Sampling (s)",
    "Average PGD Time": "PGD (s)",
    "Std PGD Time": "PGD (s)",
    "Average Loss Time": "LossTime (s)",
    "Std Loss Time": "LossTime (s)",
    "Average Total Time": "Total (s)",
    "Std Total Time": "Total (s)",
}


def latex_escape(text: str) -> str:
    return text.replace("_", "\\_").replace("%", "\\%")


def format_numeric(value: str, param_name: str = "") -> str:
    if value in ("True", "False"):
        return value
    if param_name.lower() in (
        "seed", "iter", "search_width", "min_search_width",
        "num_steps", "num_prompts", "k"
    ):
        try:
            return str(int(float(value)))
        except:
            return latex_escape(value)
    try:
        f = float(value)
        return f"{f:.4f}"
    except:
        return latex_escape(value)


def combine_avg_std(avg_val: str, std_val: str) -> str:
    if not avg_val or not std_val:
        return "0.0000±0.0000"
    try:
        a, s = float(avg_val), float(std_val)
        return f"{a:.4f}±{s:.4f}"
    except:
        return "0.0000±0.0000"


# Discover experiments
for idx, folder in enumerate(sorted(os.listdir(experiments_dir)), start=1):
    exp_id = f"exp{idx}"
    path = os.path.join(experiments_dir, folder)

    if not os.path.isdir(path):
        continue

    # --- PARAMETERS row ---
    params = {}
    pfile = os.path.join(path, "parameters.csv")
    if os.path.exists(pfile):
        with open(pfile) as cf:
            for row in csv.DictReader(cf):
                key = row["Parameter"]
                if key == "debug_output":
                    continue
                val = row["Value"]
                params[key] = val
                all_param_keys.add(key)
    parameters_rows.append({"Experiment": exp_id, **params})

    # --- METRICS from summary.csv ---
    metrics = {}
    main_summary = os.path.join(path, "summary.csv")
    if os.path.exists(main_summary):
        with open(main_summary) as cf:
            for row in csv.DictReader(cf):
                metrics[row["Metric"]] = row["Value"]

    # --- DYNAMIC AS@k from each evaluation_k*/summary.csv ---
    # find all evaluation_k* directories
    as_map = {}  # k_str -> (success, total)
    for d in os.listdir(path):
        if not d.startswith("evaluation_k"):
            continue
        k_str = d.split("_k", 1)[1]
        summary_csv = os.path.join(path, d, "summary.csv")
        if not os.path.exists(summary_csv):
            continue
        succ = tot = 0
        with open(summary_csv) as cf:
            for row in csv.DictReader(cf):
                v = row.get("success@k", "").strip().lower()
                if v in ("true", "1"):
                    succ += 1
                    tot += 1
                elif v in ("false", "0"):
                    tot += 1
        as_map[k_str] = (succ, tot)
        as_columns.add(f"AS@{k_str}")

    # --- BUILD summary_row ---
    row = {"Experiment": exp_id}
    # Loss
    row["Loss"] = combine_avg_std(
        metrics.get("Average Best Loss", ""),
        metrics.get("Std Best Loss", "")
    )
    # times
    for avg_key, col in [
        ("Average Gradient Time", "Grad (s)"),
        ("Average Sampling Time", "Sampling (s)"),
        ("Average PGD Time", "PGD (s)"),
        ("Average Loss Time", "LossTime (s)"),
        ("Average Total Time", "Total (s)")
    ]:
        std_key = avg_key.replace("Average", "Std")
        row[col] = combine_avg_std(
            metrics.get(avg_key, ""),
            metrics.get(std_key, "")
        )
    # AS@k columns
    for k_str, (succ, tot) in as_map.items():
        row[f"AS@{k_str}"] = f"{succ}/{tot}" if tot > 0 else "N/A"

    summary_rows.append(row)


# --- SORT experiments by model then index ---
combined = [{"p": p, "s": s} for p, s in zip(parameters_rows, summary_rows)]
def sort_key(item):
    m = item["p"].get("model", "").lower()
    grp = 0 if m == "llava" else 1 if m == "gemma" else 2
    num = int(item["p"]["Experiment"].replace("exp", ""))
    return grp, num

combined.sort(key=sort_key)
for new_idx, item in enumerate(combined, start=1):
    eid = f"exp{new_idx}"
    item["p"]["Experiment"] = eid
    item["s"]["Experiment"] = eid

parameters_rows = [item["p"] for item in combined]
summary_rows    = [item["s"] for item in combined]
all_param_keys.discard("seed")

# --- LaTeX PARAMETERS table ---
param_cols = ["Experiment"] + sorted(all_param_keys, key=str.lower)
latex_params = [
    r"\begin{table}[ht]",
    r"\centering",
    r"\resizebox{\textwidth}{!}{%",
    "  \\begin{tabular}{" + "l" * len(param_cols) + "}",
    "  \\hline",
    "  " + " & ".join(latex_escape(c) for c in param_cols) + r" \\",
    "  \\hline",
]
for r in parameters_rows:
    vals = [ format_numeric(r.get(c, ""), param_name=c) for c in param_cols ]
    latex_params.append("  " + " & ".join(vals) + r" \\")
latex_params += [
    "  \\hline",
    "  \\end{tabular}",
    "} % end resizebox",
    r"\caption{Experiment parameters.}",
    r"\label{tab:experiment_parameters}",
    r"\end{table}"
]

# --- LaTeX SUMMARY table ---
fixed = ["Experiment", "Loss", "Grad (s)", "Sampling (s)",
         "PGD (s)", "LossTime (s)", "Total (s)"]
# sort AS@k columns numerically
as_cols_sorted = sorted(
    as_columns,
    key=lambda x: int(x.split("@")[1]) if x.split("@")[1].isdigit() else x
)
final_cols = fixed + as_cols_sorted
final_disp = [latex_escape(c) for c in final_cols]

latex_summary = [
    r"\begin{table}[ht]",
    r"\centering",
    r"\resizebox{\textwidth}{!}{%",
    "  \\begin{tabular}{" + "l" * len(final_disp) + "}",
    "  \\hline",
    "  " + " & ".join(final_disp) + r" \\",
    "  \\hline",
]
for r in summary_rows:
    vals = [ latex_escape(r.get(c, "")) for c in final_cols ]
    latex_summary.append("  " + " & ".join(vals) + r" \\")
latex_summary += [
    "  \\hline",
    "  \\end{tabular}",
    "} % end resizebox",
    r"\caption{Summary of experiment results.}",
    r"\label{tab:experiment_summary}",
    r"\end{table}"
]

print("%% ========== TABLE: PARAMETERS ========== %%")
print("\n".join(latex_params))
print("\n%% ========== TABLE: SUMMARY ========== %%")
print("\n".join(latex_summary))


%% ========== TABLE: PARAMETERS ========== %%
\begin{table}[ht]
\centering
\resizebox{\textwidth}{!}{%
  \begin{tabular}{llllllllllllll}
  \hline
  Experiment & alpha & dynamic\_search & eps & gcg\_attack & joint\_eval & min\_search\_width & model & name & num\_prompts & num\_steps & pgd\_after\_gcg & pgd\_attack & search\_width \\
  \hline
  exp1 & 4/255 & False & 64/255 & False & False & 0 & llava & Llava - PGD Only & 20 & 600 & False & True & 0 \\
  exp2 & 0/255 & False & 0/255 & True & False & 512 & llava & Llava - GCG Only & 20 & 250 & False & False & 512 \\
  exp3 & 4/255 & False & 64/255 & True & False & 0 & llava & Llava - PGD + GCG & 20 & 250 & False & True & 512 \\
  exp4 & 4/255 & False & 64/255 & False & False & 0 & llava & Llava - PGD Only & 10 & 600 & False & True & 0 \\
  exp5 & 0/255 & False & 8/255 & False & False & 0 & llava & Llava - Auto-PGD 2 & 10 & 600 & False & True & 0 \\
  exp6 & 4/255 & False & 64/255 & True & False & 0 & llava & Llava - GCG + PGD & 20 & 250 &

In [7]:
import os
import csv
import statistics

# Directory containing experiment folders
experiments_dir = "experiments"

parameters_rows = []
summary_rows = []

all_param_keys = set()
as_columns = set()


def latex_escape(text: str) -> str:
    return text.replace("_", "\\_").replace("%", "\\%")


def format_numeric(value: str, param_name: str = "") -> str:
    if value == "True":
        return r"\cmark"
    if value == "False":
        return r"\xmark"
    if param_name.lower() in {
        "seed", "iter", "search_width", "min_search_width",
        "num_steps", "num_prompts", "k"
    }:
        try:
            return str(int(float(value)))
        except:
            return latex_escape(value)
    try:
        return f"{float(value):.4f}"
    except:
        return latex_escape(value)


def combine_avg_std(avg, std, d=4) -> str:
    default = f"{0:.{d}f}±{0:.{d}f}"
    if not avg or not std:
        return default
    try:
        return f"{float(avg):.{d}f}±{float(std):.{d}f}"
    except:
        return default


for idx, folder in enumerate(sorted(os.listdir(experiments_dir)), start=1):
    exp_id = f"exp{idx}"
    path = os.path.join(experiments_dir, folder)
    if not os.path.isdir(path):
        continue

    # PARAMETERS
    params = {}
    pfile = os.path.join(path, "parameters.csv")
    if os.path.exists(pfile):
        with open(pfile) as cf:
            for row in csv.DictReader(cf):
                k = row["Parameter"]
                if k == "debug_output":
                    continue
                v = row["Value"]
                params[k] = v
                all_param_keys.add(k)
    parameters_rows.append({"Experiment": exp_id, **params})

    # METRICS
    metrics = {}
    mfile = os.path.join(path, "summary.csv")
    if os.path.exists(mfile):
        with open(mfile) as cf:
            for row in csv.DictReader(cf):
                metrics[row["Metric"]] = row["Value"]

    # DYNAMIC AS@k
    eval_root = os.path.join(path, "evaluation")
    as_map = {}
    if os.path.isdir(eval_root):
        for d in sorted(os.listdir(eval_root)):
            if not d.startswith("evaluation_k"):
                continue
            k_str = d.split("_k", 1)[1]
            scsv = os.path.join(eval_root, d, "summary.csv")
            if not os.path.exists(scsv):
                continue
            succ = tot = 0
            with open(scsv) as cf:
                for row in csv.DictReader(cf):
                    v = row.get("success@k", "").strip().lower()
                    if v in ("true", "1"):
                        succ += 1
                        tot += 1
                    elif v in ("false", "0"):
                        tot += 1
            as_map[k_str] = (succ, tot)
            as_columns.add(f"AS@{k_str}")

    # BUILD summary row
    row = {"Experiment": exp_id}
    row["Loss"] = combine_avg_std(
        metrics.get("Average Best Loss", ""),
        metrics.get("Std Best Loss", "")
    )
    row["Total (s)"] = combine_avg_std(
        metrics.get("Average Total Time", ""),
        metrics.get("Std Total Time", "")
    )
    for k_str, (succ, tot) in as_map.items():
        row[f"AS@{k_str}"] = f"{succ}/{tot}" if tot else "N/A"
    summary_rows.append(row)


# SORT
combined = [{"p": p, "s": s} for p, s in zip(parameters_rows, summary_rows)]
def sort_key(item):
    m = item["p"].get("model", "").lower()
    grp = 0 if m == "llava" else 1 if m == "gemma" else 2
    num = int(item["p"]["Experiment"].replace("exp", ""))
    return grp, num

combined.sort(key=sort_key)
for new_idx, item in enumerate(combined, start=1):
    eid = f"exp{new_idx}"
    item["p"]["Experiment"] = eid
    item["s"]["Experiment"] = eid

parameters_rows = [it["p"] for it in combined]
summary_rows    = [it["s"] for it in combined]
all_param_keys.discard("seed")

# LaTeX PARAMETERS
param_cols = ["Experiment"] + sorted(all_param_keys, key=str.lower)
latex_params = [
    r"\begin{table}[ht]",
    r"\centering",
    r"\resizebox{\textwidth}{!}{%",
    "  \\begin{tabular}{" + "l"*len(param_cols) + "}",
    "  \\hline",
    "  " + " & ".join(latex_escape(c) for c in param_cols) + r" \\",
    "  \\hline",
]
for r in parameters_rows:
    vals = [format_numeric(r.get(c, ""), param_name=c) for c in param_cols]
    latex_params.append("  " + " & ".join(vals) + r" \\")
latex_params += [
    "  \\hline",
    "  \\end{tabular}",
    "} % end resizebox",
    r"\caption{Experiment parameters.}",
    r"\label{tab:experiment_parameters}",
    r"\end{table}"
]

# LaTeX SUMMARY
fixed = ["Experiment", "Loss", "Total (s)"]
as_cols_sorted = sorted(
    as_columns,
    key=lambda x: int(x.split("@")[1]) if x.split("@")[1].isdigit() else x
)
final_cols = fixed + as_cols_sorted
final_disp = [latex_escape(c) for c in final_cols]

latex_summary = [
    r"\begin{table}[ht]",
    r"\centering",
    r"\resizebox{\textwidth}{!}{%",
    "  \\begin{tabular}{" + "l"*len(final_disp) + "}",
    "  \\hline",
    "  " + " & ".join(final_disp) + r" \\",
    "  \\hline",
]
for r in summary_rows:
    vals = [latex_escape(r.get(c, "")) for c in final_cols]
    latex_summary.append("  " + " & ".join(vals) + r" \\")
latex_summary += [
    "  \\hline",
    "  \\end{tabular}",
    "} % end resizebox",
    r"\caption{Summary of experiment results.}",
    r"\label{tab:experiment_summary}",
    r"\end{table}"
]

print("%% ========== TABLE: PARAMETERS ========== %%")
print("\n".join(latex_params))
print("\n%% ========== TABLE: SUMMARY ========== %%")
print("\n".join(latex_summary))


%% ========== TABLE: PARAMETERS ========== %%
\begin{table}[ht]
\centering
\resizebox{\textwidth}{!}{%
  \begin{tabular}{llllllllllllll}
  \hline
  Experiment & alpha & dynamic\_search & eps & gcg\_attack & joint\_eval & min\_search\_width & model & name & num\_prompts & num\_steps & pgd\_after\_gcg & pgd\_attack & search\_width \\
  \hline
  exp1 & 4/255 & \xmark & 64/255 & \xmark & \xmark & 0 & llava & Llava - PGD Only & 20 & 600 & \xmark & \cmark & 0 \\
  exp2 & 0/255 & \xmark & 0/255 & \cmark & \xmark & 512 & llava & Llava - GCG Only & 20 & 250 & \xmark & \xmark & 512 \\
  exp3 & 4/255 & \xmark & 64/255 & \cmark & \xmark & 0 & llava & Llava - PGD + GCG & 20 & 250 & \xmark & \cmark & 512 \\
  exp4 & 4/255 & \xmark & 64/255 & \xmark & \xmark & 0 & llava & Llava - PGD Only & 10 & 600 & \xmark & \cmark & 0 \\
  exp5 & 0/255 & \xmark & 8/255 & \xmark & \xmark & 0 & llava & Llava - Auto-PGD 2 & 10 & 600 & \xmark & \cmark & 0 \\
  exp6 & 4/255 & \xmark & 64/255 & \cmark & \xmark & 0 & lla